In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="8"

In [2]:
import gc
import numpy as np
from itertools import product
import scipy.io
from scipy.ndimage import convolve
from scipy.signal import convolve2d
from skimage.color import rgb2gray
from scipy.ndimage import convolve, gaussian_filter
from numpy.fft import fft2, ifft2, fftshift
import scipy.io
from scipy import signal
import cv2
import numpy as np
from scipy.ndimage import zoom
from scipy.signal import convolve2d
from PIL import Image
from scipy.ndimage import convolve
from scipy.ndimage import gaussian_filter, maximum_filter
from scipy.sparse import spdiags, eye
from scipy.sparse.linalg import spsolve
from skimage.color import rgb2gray
from sklearn.neighbors import KDTree
from scipy.signal import fftconvolve
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import scipy.sparse as sparse
from sklearn.metrics import mean_squared_error as mse

In [3]:
def convolve_rgb(image, kernel):
    border_size = 1
    foggy_image_with_border = cv2.copyMakeBorder(image,
                                                 top=border_size,
                                                 bottom=border_size,
                                                 left=border_size,
                                                 right=border_size,
                                                 borderType=cv2.BORDER_WRAP)

    # Apply the filter
    temp = cv2.filter2D(foggy_image_with_border, -1, kernel)

    # Remove the added border
    temp = temp[border_size:-border_size, border_size:-border_size]

    return temp

def rgb2gray(rgb_image):
    """
    Convert an RGB image to a grayscale image.
    :param rgb_image: Input RGB image as a NumPy array.
    :return: Grayscale image as a NumPy array.
    """
    # Check if the input image is a 3D array (RGB image)
    if rgb_image.ndim != 3 or rgb_image.shape[2] != 3:
        raise ValueError("Input must be an RGB image")

    # Define the coefficients for the RGB channels
    coeffs = np.array([0.2989, 0.5870, 0.1140])

    # Convert the image to grayscale
    gray_image = np.dot(rgb_image[..., :3], coeffs)

    return gray_image

def convolve_rgb(image, kernel):
    border_size = 1
    foggy_image_with_border = cv2.copyMakeBorder(image,
                                                 top=border_size,
                                                 bottom=border_size,
                                                 left=border_size,
                                                 right=border_size,
                                                 borderType=cv2.BORDER_WRAP)

    # Apply the filter
    temp = cv2.filter2D(foggy_image_with_border, -1, kernel)

    # Remove the added border
    temp = temp[border_size:-border_size, border_size:-border_size]

    return temp

def gradient_weight(I):
    if len(I.shape) == 3 and I.shape[2] == 3:  # If the image has RGB channels
        I = np.dot(I[..., :3], [0.2989, 0.5870, 0.1140])  # Convert to grayscale using standard weights

    lambda_val = 10
    f1 = np.array([[0, 0, 0], [1, -1, 0], [0, 0, 0]])
    f2 = np.array([[0, 1, 0], [0, -1, 0], [0, 0, 0]])

    Gx = signal.convolve2d(I, f1, mode='same')
    Gy = signal.convolve2d(I, f2, mode='same')

    ax = np.exp(-lambda_val * np.abs(Gx))
    thx = Gx < 0.01
    ax[thx] = 0
    weight_x = 1 + ax

    ay = np.exp(-lambda_val * np.abs(Gy))
    thy = Gy < 0.01
    ay[thy] = 0
    weight_y = 1 + ay

    return weight_x, weight_y


def psf2otf(psf, shape):
    psf = np.pad(psf, [(0, shape[0] - psf.shape[0]), (0, shape[1] - psf.shape[1])], mode='constant')
    for axis, axis_size in enumerate(psf.shape):
        psf = np.roll(psf, -axis_size // 2, axis=axis)
    otf = fft2(psf.T).T
    return otf

In [4]:
def preprocess_for_training(input_img):
    # foggy_image_path = './Test_Data/V_08_01_0038.jpg' # This is relative path which needs to be updated.
    # mat_contents = scipy.io.loadmat('./mat_data_file/V_08_01_0038.mat')
    # ii = mat_contents['ii']
    # A = mat_contents['A']
    ii = 5

    # Read the image using OpenCV
    adjust_fog_removal = 2
    brightness = 2

    # Convert image from uint8 to float32 (similar to im2double)

    I = input_img.copy()
    alpha = 20000
    beta = 0.1
    gamma = 10

    alpha = float(alpha)
    ii = float(ii)
    beta = float(beta)
    gamma = float(gamma)
    I = np.clip(I, 0, 1)
    gray = rgb2gray(I)
    H, W, D = I.shape

    weight_x, weight_y = gradient_weight(I)
    # f1 = np.array([[1, -1]])
    # f2 = np.array([[1], [-1]])
    f1 = np.array([[0, 0, 0], [1, -1, 0], [0, 0, 0]])
    f2 = np.array([[0, 1, 0], [0, -1, 0], [0, 0, 0]])
    f4 = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]])

    gray = rgb2gray(I)
    I_filt = gaussian_filter(gray, sigma=10)
    delta_I = I - I_filt[..., np.newaxis]

    otfFx = psf2otf(f1, (H, W))
    otfFy = psf2otf(f2, (H, W))
    otfL = psf2otf(f4, (H, W))

    fft_double_laplace = np.abs(otfL) ** 2
    fft_double_grad = np.abs(otfFx) ** 2 + np.abs(otfFy) ** 2

    if D > 1:
        fft_double_grad = np.repeat(fft_double_grad[:, :, np.newaxis], D, axis=2)
        fft_double_laplace = np.repeat(fft_double_laplace[:, :, np.newaxis], D, axis=2)
        weight_x = np.repeat(weight_x[:, :, np.newaxis], D, axis=2)
        weight_y = np.repeat(weight_y[:, :, np.newaxis], D, axis=2)

    F = np.zeros_like(I)
    N = np.zeros_like(I)
    gray = rgb2gray(I)
    Ix = np.zeros_like(I)
    Iy = np.zeros_like(I)

    for channel in range(I.shape[2]):
        Ix[:, :, channel] = signal.convolve2d(I[:, :, channel], f1, mode='same')
        Iy[:, :, channel] = signal.convolve2d(I[:, :, channel], f2, mode='same')

    Normin_I = fft2((np.concatenate((Ix[:, -1:, :] - Ix[:, :1, :], -np.diff(Ix, 1, 1)), axis=1) +
                     np.concatenate((Iy[-1:, :, :] - Iy[:1, :, :], -np.diff(Iy, 1, 0)), axis=0)).T).T
    Denormin_N = gamma + alpha * fft_double_laplace + beta
    Normin_gI = fft_double_laplace * fft2(I.T).T

    return(F, I, N, Normin_I, Normin_gI, Denormin_N, Ix, Iy, fft_double_grad, fft_double_laplace, weight_x, weight_y, delta_I)

In [5]:
from glob import glob
from tqdm import tqdm

In [6]:
paths = glob('./Test_Data/*')
len(paths)

70

In [7]:
# F_list = []
# I_list = []
# N_list = []
# Normin_I_list = []
# Normin_gI_list = []
# Denormin_N_list = []
# Ix_list = []
# Iy_list = []
# fft_double_grad_list = []
# fft_double_laplace_list = []
# weight_x_list = []
# weight_y_list = []
# delta_I_list = []

# for i in tqdm(range(len(paths))):
#     foggy_image_path = paths[i]
#     foggy_image = cv2.imread(foggy_image_path)
#     foggy_image = cv2.cvtColor(foggy_image, cv2.COLOR_BGR2RGB)
#     input_img = foggy_image.astype(np.float32) / foggy_image.max()
#     F = preprocess_for_training(input_img)
#     inputs = F.copy()
#     inputs = np.array([np.moveaxis(inputs,-1,0)])
#     inputs = torch.from_numpy(inputs).float()
#     inputs = inputs
#     outputs = model(inputs)
#     out = np.moveaxis(outputs.cpu().detach().numpy()[0,:,:,:],0,-1)
#     out = out-np.min(out)
#     out = out/np.max(out)
#     out2 = preprocess_for_training(out)
#     input_images.append(out)

In [8]:
import torch
import torch.nn as nn
import torchvision

In [9]:
def get_batchnorm_layer(opts):
    if opts.norm_layer == "batch":
        norm_layer = nn.BatchNorm2d
    elif opts.layer == "spectral_instance":
        norm_layer = nn.InstanceNorm2d
    else:
        print("not implemented")
        exit()
    return norm_layer

def get_conv2d_layer(in_c, out_c, k, s, p=0, dilation=1, groups=1):
    return nn.Conv2d(in_channels=in_c,
                    out_channels=out_c,
                    kernel_size=k,
                    stride=s,
                    padding=p,dilation=dilation, groups=groups)

def get_deconv2d_layer(in_c, out_c, k=1, s=1, p=1):
    return nn.Sequential(
        nn.Upsample(scale_factor=2, mode="bilinear"),
        nn.Conv2d(
            in_channels=in_c,
            out_channels=out_c,
            kernel_size=k,
            stride=s,
            padding=p
        )
    )

class Identity(nn.Module):

    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

In [10]:
class Decom(nn.Module):
    def __init__(self):
        super().__init__()
        self.decom = nn.Sequential(
            get_conv2d_layer(in_c=3, out_c=32, k=3, s=1, p=1),
            nn.LeakyReLU(0.2, inplace=True),
            get_conv2d_layer(in_c=32, out_c=32, k=3, s=1, p=1),
            nn.LeakyReLU(0.2, inplace=True),
            get_conv2d_layer(in_c=32, out_c=32, k=3, s=1, p=1),
            nn.LeakyReLU(0.2, inplace=True),
            get_conv2d_layer(in_c=32, out_c=4, k=3, s=1, p=1),
            nn.ReLU()
        )

    def forward(self, input):
        output = self.decom(input)
        R = output[:, 0:3, :, :]
        L = output[:, 3:4, :, :]
        return R, L

In [69]:
import torch.nn as nn
import torch
from torch.nn.modules.linear import Identity
import math
import torch.nn.functional as Func

In [12]:
class HalfDnCNNSE(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = get_conv2d_layer(in_c=3, out_c=32, k=3, s=1, p=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = get_conv2d_layer(in_c=1, out_c=32, k=3, s=1, p=1)
        self.relu2 = nn.ReLU(inplace=True)
        self.se_layer = SELayer(channel=64)
        self.conv3 = get_conv2d_layer(in_c=64, out_c=64, k=3, s=1, p=1)
        self.relu3 = nn.ReLU(inplace=True)
        self.conv4 = get_conv2d_layer(in_c=64, out_c=64, k=3, s=1, p=1)
        self.relu4 = nn.ReLU(inplace=True)
        self.conv5 = get_conv2d_layer(in_c=64, out_c=64, k=3, s=1, p=1)
        self.relu5 = nn.ReLU(inplace=True)
        self.conv6 = get_conv2d_layer(in_c=64, out_c=64, k=3, s=1, p=1)
        self.relu6 = nn.ReLU(inplace=True)
        self.conv7 = get_conv2d_layer(in_c=64, out_c=64, k=3, s=1, p=1)
        self.relu7 = nn.ReLU(inplace=True)

        self.conv8 = get_conv2d_layer(in_c=64, out_c=3, k=3, s=1, p=1)

    def forward(self, r, l):
        r_fs = self.relu1(self.conv1(r))
        l_fs = self.relu2(self.conv2(l))
        inf = torch.cat([r_fs, l_fs], dim=1)
        se_inf = self.se_layer(inf)
        x1 = self.relu3(self.conv3(se_inf))
        x2 = self.relu4(self.conv4(x1))
        x3 = self.relu5(self.conv5(x2))
        x4 = self.relu6(self.conv6(x3))
        x5 = self.relu7(self.conv7(x4))
        n = self.conv8(x5)
        r_restore = r + n
        return r_restore

class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

In [13]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.model1 = Decom()
        self.model2 = HalfDnCNNSE()

    def forward(self, x):
        x1,x2 = self.model1(x)
        x3 = self.model2(x1,x2)
        return x3


net1 = Net()
net2 = Net()
net3 = Net()

# Losses

In [70]:
class L_color(nn.Module):

    def __init__(self):
        super(L_color, self).__init__()

    def forward(self, x ):

        b,c,h,w = x.shape

        mean_rgb = torch.mean(x,[2,3],keepdim=True)
        mr,mg, mb = torch.split(mean_rgb, 1, dim=1)
        Drg = torch.pow(mr-mg,2)
        Drb = torch.pow(mr-mb,2)
        Dgb = torch.pow(mb-mg,2)
        k = torch.pow(torch.pow(Drg,2) + torch.pow(Drb,2) + torch.pow(Dgb,2),0.5)


        return k

			
class L_spa(nn.Module):

    def __init__(self):
        super(L_spa, self).__init__()
        # print(1)kernel = torch.FloatTensor(kernel).unsqueeze(0).unsqueeze(0)
        kernel_left = torch.FloatTensor( [[0,0,0],[-1,1,0],[0,0,0]]).unsqueeze(0).unsqueeze(0)
        kernel_right = torch.FloatTensor( [[0,0,0],[0,1,-1],[0,0,0]]).unsqueeze(0).unsqueeze(0)
        kernel_up = torch.FloatTensor( [[0,-1,0],[0,1, 0 ],[0,0,0]]).unsqueeze(0).unsqueeze(0)
        kernel_down = torch.FloatTensor( [[0,0,0],[0,1, 0],[0,-1,0]]).unsqueeze(0).unsqueeze(0)
        self.weight_left = nn.Parameter(data=kernel_left, requires_grad=False)
        self.weight_right = nn.Parameter(data=kernel_right, requires_grad=False)
        self.weight_up = nn.Parameter(data=kernel_up, requires_grad=False)
        self.weight_down = nn.Parameter(data=kernel_down, requires_grad=False)
        self.pool = nn.AvgPool2d(4)
    def forward(self, org , enhance ):
        b,c,h,w = org.shape

        org_mean = torch.mean(org,1,keepdim=True)
        enhance_mean = torch.mean(enhance,1,keepdim=True)

        org_pool =  self.pool(org_mean)			
        enhance_pool = self.pool(enhance_mean)	

        weight_diff =torch.max(torch.FloatTensor([1]) + 10000*torch.min(org_pool - torch.FloatTensor([0.3]),torch.FloatTensor([0])),torch.FloatTensor([0.5]))
        E_1 = torch.mul(torch.sign(enhance_pool - torch.FloatTensor([0.5])) ,enhance_pool-org_pool)


        D_org_letf = Func.conv2d(org_pool , self.weight_left, padding=1)
        D_org_right = Func.conv2d(org_pool , self.weight_right, padding=1)
        D_org_up = Func.conv2d(org_pool , self.weight_up, padding=1)
        D_org_down = Func.conv2d(org_pool , self.weight_down, padding=1)

        D_enhance_letf = Func.conv2d(enhance_pool , self.weight_left, padding=1)
        D_enhance_right = Func.conv2d(enhance_pool , self.weight_right, padding=1)
        D_enhance_up = Func.conv2d(enhance_pool , self.weight_up, padding=1)
        D_enhance_down = Func.conv2d(enhance_pool , self.weight_down, padding=1)

        D_left = torch.pow(D_org_letf - D_enhance_letf,2)
        D_right = torch.pow(D_org_right - D_enhance_right,2)
        D_up = torch.pow(D_org_up - D_enhance_up,2)
        D_down = torch.pow(D_org_down - D_enhance_down,2)
        E = (D_left + D_right + D_up +D_down)
        # E = 25*(D_left + D_right + D_up +D_down)

        return E
class L_exp(nn.Module):

    def __init__(self,patch_size,mean_val):
        super(L_exp, self).__init__()
        # print(1)
        self.pool = nn.AvgPool2d(patch_size)
        self.mean_val = mean_val
    def forward(self, x ):

        b,c,h,w = x.shape
        x = torch.mean(x,1,keepdim=True)
        mean = self.pool(x)

        d = torch.mean(torch.pow(mean- torch.FloatTensor([self.mean_val] ),2))
        return d
        
class L_TV(nn.Module):
    def __init__(self,TVLoss_weight=1):
        super(L_TV,self).__init__()
        self.TVLoss_weight = TVLoss_weight

    def forward(self,x):
        batch_size = x.size()[0]
        h_x = x.size()[2]
        w_x = x.size()[3]
        count_h =  (x.size()[2]-1) * x.size()[3]
        count_w = x.size()[2] * (x.size()[3] - 1)
        h_tv = torch.pow((x[:,:,1:,:]-x[:,:,:h_x-1,:]),2).sum()
        w_tv = torch.pow((x[:,:,:,1:]-x[:,:,:,:w_x-1]),2).sum()
        return self.TVLoss_weight*2*(h_tv/count_h+w_tv/count_w)/batch_size
class Sa_Loss(nn.Module):
    def __init__(self):
        super(Sa_Loss, self).__init__()
        # print(1)
    def forward(self, x ):
        # self.grad = np.ones(x.shape,dtype=np.float32)
        b,c,h,w = x.shape
        # x_de = x.cpu().detach().numpy()
        r,g,b = torch.split(x , 1, dim=1)
        mean_rgb = torch.mean(x,[2,3],keepdim=True)
        mr,mg, mb = torch.split(mean_rgb, 1, dim=1)
        Dr = r-mr
        Dg = g-mg
        Db = b-mb
        k =torch.pow( torch.pow(Dr,2) + torch.pow(Db,2) + torch.pow(Dg,2),0.5)
        # print(k)
        

        k = torch.mean(k)
        return k

In [71]:
L_color = L_color()
L_spa = L_spa()

L_exp = L_exp(16,0.6)
L_TV = L_TV()

In [72]:
import torch
import torch.fft
import torch.nn.functional as Ff

def torch_convolve_rgb(image, kernel):
    # PyTorch's conv2d expects a 4D tensor of shape (batch_size, channels, height, width)
    # Adjust the input image tensor to have channels as the second dimension
    if image.dim() == 3:  # If the image is of shape (H, W, C)
        image = image.permute(2, 0, 1)  # Change to (C, H, W)
    image = image.unsqueeze(0)  # Add the batch dimension

    # Adjusting kernel shape for PyTorch (out_channels, in_channels, height, width)
    # Assuming kernel is a square matrix
    kernel_size = kernel.shape[0]
    kernel = kernel.expand(3, 1, kernel_size, kernel_size)

    # Convolution
    convolved = Ff.conv2d(image, kernel, padding=1, groups=3)
    convolved = convolved.squeeze(0)  # Remove the batch dimension

    # Rearrange the tensor back to (H, W, C) if necessary
    if convolved.dim() == 3:
        convolved = convolved.permute(1, 2, 0)

    return convolved
    

def create_gaussian_kernel(sigma, truncate):
    radius = int(truncate * sigma + 0.5)
    size = 2 * radius + 1
    x = torch.arange(-radius, radius + 1, dtype=torch.float32)
    kernel = torch.exp(-0.5 / (sigma ** 2) * x ** 2)
    kernel /= kernel.sum()
    return kernel

def gaussian_filter_pytorch(input, sigma, truncate=4.0):
    kernel = create_gaussian_kernel(sigma, truncate)
    kernel_size = kernel.shape[0]
    kernel = kernel.view(1, 1, *kernel.shape)

    input_channels = input.size(1)
    kernel = kernel.repeat(input_channels, 1, 1, 1)

    padding = kernel_size // 2
    filtered = Ff.conv2d(input, kernel, padding=padding, groups=input_channels)

    # Calculate cropping to match input dimensions
    crop_h = (filtered.size(2) - input.size(2)) // 2
    crop_w = (filtered.size(3) - input.size(3)) // 2
    filtered_cropped = filtered[:, :, crop_h:filtered.size(2)-crop_h, crop_w:filtered.size(3)-crop_w]

    return filtered_cropped

def compute_decomposition_step_dev(stage,F, I, N, Normin_I,  Normin_gI, Denormin_N, Ix, Iy, fft_double_grad, fft_double_laplace, alpha, beta, gamma, weight_x, weight_y, delta_I, H, W, Dd, i, ii):
    if(stage==0):
        F = torch.from_numpy(F).float()
        N = torch.from_numpy(N).float()        
    I = torch.from_numpy(I).float()
    Normin_I = torch.from_numpy(Normin_I).cfloat()
    Normin_gI = torch.from_numpy(Normin_gI).cfloat()
    Ix = torch.from_numpy(Ix).float()
    Iy = torch.from_numpy(Iy).float()
    fft_double_grad = torch.from_numpy(fft_double_grad).float()
    fft_double_laplace = torch.from_numpy(fft_double_laplace).float()
    weight_x = torch.from_numpy(weight_x).float()
    weight_y = torch.from_numpy(weight_y).float()
    delta_I = torch.from_numpy(delta_I).float()
    Denormin_N = torch.from_numpy(Denormin_N)

    
    lambda_ = min(2 ** (ii + i), 10 ** 5)
    Denormin_F = lambda_ * fft_double_grad + alpha * fft_double_laplace + beta
       
    f1_tensor = torch.tensor([[0, 0, 0], [1, -1, 0], [0, 0, 0]], dtype=torch.float32)
    f2_tensor = torch.tensor([[0, 1, 0], [0, -1, 0], [0, 0, 0]], dtype=torch.float32)


    # Update q using the tensor versions of the filters
    qx = -torch_convolve_rgb(F, f1_tensor) - Ix
    qy = -torch_convolve_rgb(F, f2_tensor) - Iy
        
    qx = torch.sign(qx) * torch.clamp(torch.abs(qx) - weight_x / lambda_, min=0)
    qy = torch.sign(qy) * torch.clamp(torch.abs(qy) - weight_y / lambda_, min=0)


    Normin_q_x = torch.cat((qx[:, -1:, :] - qx[:,:1, :], -torch.diff(qx, dim=1)), dim=1)
    Normin_q_y = torch.cat((qy[-1:, :, :] - qy[:1, :], -torch.diff(qy, dim=0)), dim=0)
    Normin_q = Normin_q_x + Normin_q_y

    N_expanded = N.unsqueeze(2) if N.dim() == 2 else N
    # print("N_expanded dimensions:", N_expanded.shape)
    Normin_gN = fft_double_laplace * torch.fft.fft2(N_expanded.permute(2, 0, 1)).permute(1, 2, 0)

    FF = (lambda_ * (Normin_I + torch.fft.fft2(Normin_q.permute(2, 0, 1)).permute(1, 2, 0)) + alpha * (Normin_gI - Normin_gN) + beta * torch.fft.fft2((delta_I - N_expanded).permute(2, 0, 1)).permute(1, 2, 0)) / Denormin_F
    F = torch.fft.ifft2(FF.permute(2, 0, 1)).permute(1, 2, 0).real


    Normin_F = fft_double_laplace * torch.fft.fft2(F.permute(2, 0, 1)).permute(1, 2, 0)
    B = torch.fft.fft2((delta_I - F).permute(2, 0, 1)).permute(1, 2, 0)
    NN = (alpha * (Normin_gI - Normin_F) + beta * B) / Denormin_N
    # NN = torch.from_numpy(NN).float()
    N = torch.fft.ifft2(NN.permute(2, 0, 1)).permute(1, 2, 0).real
    N = N.squeeze() if N.dim() == 2 else N

    for c in range(Dd):
        Ft = F[:, :, c]
        q = torch.numel(Ft)
        for k in range(500):
            m = torch.sum(Ft[Ft < 0])
            n = torch.sum(Ft[Ft > 1] - 1)
            dt = (m + n) / q
            if torch.abs(dt) < 1 / q:
                break
            Ft = Ft - dt
        F[:, :, c] = Ft

    F = torch.abs(F)
    F = torch.clamp(F, max=1)
    N = torch.clamp(N, max=1, min=0)
    N = torch.mean(N, dim=2)
    
    # # Compute G
    G = torch.abs(I - F - N.unsqueeze(2))
    G = torch.min(G, dim=2).values
    
    # # Apply Gaussian filter (assuming gaussian_kernel function is defined and returns a PyTorch tensor)
    G = G.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions if needed
    G = G.float()
    G = gaussian_filter_pytorch(G, sigma=3)
    G = G.squeeze(0).squeeze(0)

    
    G_expanded = G.unsqueeze(-1)  # Shape becomes (358, 640, 1)
    N_expanded = N.unsqueeze(-1)  # Shape becomes (358, 640, 1)
    
    # Perform the operation
    F = torch.abs(I - G_expanded - N_expanded)
    
    # Replace zeros with 0.001
    F[F == 0] = 0.001
    
    # delta_I = delta_I.numpy()
    # Normin_I = Normin_I.numpy()
    # Normin_q = Normin_q.numpy()
    # weight_x = weight_x.numpy()
    # weight_y = weight_y.numpy()
    # # F = F.numpy()
    # Ix = Ix.numpy()
    # Iy = Iy.numpy()
    # qx = qx.numpy()
    # qy = qy.numpy()
    # fft_double_grad = fft_double_grad.numpy()
    # fft_double_laplace = fft_double_laplace.numpy()
    # Denormin_F = Denormin_F.numpy()
    # # N = N.numpy()
    # N_expanded = N_expanded.numpy()
    # Normin_gN = Normin_gN.numpy()
    # Normin_gI = Normin_gI.numpy()
    # I = I.numpy()
    # G = G.numpy() 

    return F, G, N

(614, 1000, 3)


# Training

In [17]:
optimizer = torch.optim.SGD(list(net1.parameters())+list(net2.parameters())+list(net3.parameters()), lr=0.001, momentum=0.9)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# net.to(device)

In [18]:
from IPython.display import clear_output

In [ ]:
epoch = 0
step = 0
losses = []
for i in range(20):
    for j in range(len(paths)-10):
        clear_output(wait=True)
        print('epoch : ',epoch)
        print('step : ',step)
        if(len(losses)!=0):
            print('loss : ',losses[0],' to ',losses[-1]) 
        step = step+1
        optimizer.zero_grad()
        foggy_image_path = paths[j]
        foggy_image = cv2.imread(foggy_image_path)
        foggy_image = cv2.cvtColor(foggy_image, cv2.COLOR_BGR2RGB)
        input_img = foggy_image.astype(np.float32) / foggy_image.max()
        H, W, D = input_img.shape
        alpha = torch.tensor([20000])
        ii = torch.tensor([5])
        beta = torch.tensor([0.1])
        gamma = torch.tensor([10])
        F, I, N, Normin_I, Normin_gI, Denormin_N, Ix, Iy, fft_double_grad, fft_double_laplace, weight_x, weight_y, delta_I = preprocess_for_training(input_img)
        F, G, N = compute_decomposition_step_dev(0,F, I, N, Normin_I, Normin_gI, Denormin_N, Ix, Iy, fft_double_grad, fft_double_laplace, 
                                             alpha, beta, gamma, weight_x, weight_y, delta_I, H, W, D, 0, ii)

        F = torch.unsqueeze(torch.moveaxis(F, -1, 0),0).float()
        F = net1(F)
        F = torch.moveaxis(F[0,:,:,:], 0, -1)
        F = F.float()
        N = N.float()
        F, G, N = compute_decomposition_step_dev(1,F, I, N, Normin_I, Normin_gI, Denormin_N, Ix, Iy, fft_double_grad, fft_double_laplace, 
                                             alpha, beta, gamma, weight_x, weight_y, delta_I, H, W, D, 1, ii)
        F = torch.unsqueeze(torch.moveaxis(F, -1, 0),0).float()
        F = net2(F)
        F = torch.moveaxis(F[0,:,:,:], 0, -1)
        F = F.float()
        N = N.float()
        F, G, N = compute_decomposition_step_dev(2,F, I, N, Normin_I, Normin_gI, Denormin_N, Ix, Iy, fft_double_grad, fft_double_laplace, 
                                             alpha, beta, gamma, weight_x, weight_y, delta_I, H, W, D, 2, ii)
        F = torch.unsqueeze(torch.moveaxis(F, -1, 0),0).float()
        F1 = net3(F)

        loss_spa = torch.mean(L_spa(F1, F))
        loss_col = 5*torch.mean(L_color(F1))
        loss_exp = 10*torch.mean(L_exp(F1))
        loss =  loss_spa + loss_col+loss_exp
        loss.backward()
        optimizer.step()
        losses.append(loss)
    step = 0
    epoch = epoch+1

epoch :  0
step :  8
loss :  tensor(3.2215, grad_fn=<AddBackward0>)  to  tensor(1.2272, grad_fn=<AddBackward0>)


In [50]:
N.shape

torch.Size([614, 1000, 3])

In [28]:
F.shape

torch.Size([614, 1000, 3])